<a href="https://colab.research.google.com/github/olakoja/DA-MUS/blob/main/%D0%A0%D0%B0%D1%81%D1%87%D1%91%D1%82_%D0%BD%D0%B5%D0%BA%D0%BE%D1%82%D0%BE%D1%80%D1%8B%D1%85_%D1%87%D0%B8%D1%81%D0%BB%D0%B5%D0%BD%D0%BD%D1%8B%D1%85_%D0%BF%D0%BE%D0%BA%D0%B0%D0%B7%D0%B0%D1%82%D0%B5%D0%BB%D0%B5%D0%B9_%D0%BF%D0%BE%D1%82%D0%B5%D0%BD%D1%86%D0%B8%D0%B0%D0%BB%D0%B0_%D1%80%D0%B0%D0%B7%D0%B2%D0%B8%D1%82%D0%B8%D1%8F_%D0%BD%D0%B5_%D0%BC%D0%B5%D1%81%D1%82_%D0%BF%D0%B5%D1%80%D0%B2%D0%BE%D0%B9_%D0%BA%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D0%B8ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Импорт бибилиотек

In [ ]:
import os
import numpy as np
import geopandas as gpd
from shapely.ops import unary_union
from tqdm import tqdm
import time
import pandas as pd
import math

# Определение числовых параметров для расчёта

In [ ]:
# Площадки для малышей (до 3 лет) площадью 75 м^2
S = 75  # Площадь в м^2
CONFIG = {
    'S': S,
    # Параметры обработки полигонов
    'inner_buffer': 20,                            # Величина внутреннего буфера (м)
    'buffer_radius': math.sqrt(S / math.pi),       # Радиус буфера вокруг точек (м)

    # Параметры сетки
    'step': 2 * math.sqrt(S / math.pi) + 1,        # Шаг сетки (м)
    'offset_step': 0.5,                            # Шаг смещения (м)

    # Группировка данных
    'density_bins': [0, 0.01, 0.1, 0.2, float('inf')],  # 4 границы = 4 интервала
    'density_labels': ['0-0.01', '0.01-0.1', '0.1-0.2', '0.2+'],  # 4 метки для 4 интервалов
    'target_group': '0-0.01',             # Обновлено для соответствия количеству бинов

    # Системные настройки
    'output_crs': 'EPSG:3857'                      # Система координат для вывода
}

In [ ]:
# Площадки для детей от 3 до 7 лет площадью 150 м^2
S = 150  # Площадь в м^2
CONFIG = {
    'S': S,
    # Параметры обработки полигонов
    'inner_buffer': 20,                           # Величина внутреннего буфера (м)
    'buffer_radius': math.sqrt(S / math.pi),      # Радиус буфера вокруг точек (м)

    # Параметры сетки
    'step': 2 * math.sqrt(S / math.pi) + 1,      # Шаг сетки (м)
    'offset_step': 0.5,                           # Шаг смещения (м)

    # Группировка данных
    'density_bins': [0, 0.01, float('inf')],
    'density_labels': ['0-0.01', '0.01+'],        # Обновлено для соответствия количеству бинов

    # Системные настройки
    'output_crs': 'EPSG:3857'                     # Система координат для вывода
}

In [ ]:
# Площадки для детей от 7 лет площадью 300 м^2
S = 300  # Площадь в м^2
CONFIG = {
    'S': S,
    # Параметры обработки полигонов
    'inner_buffer': 20,                            # Величина внутреннего буфера (м)
    'buffer_radius': math.sqrt(S / math.pi),       # Радиус буфера вокруг точек (м)

    # Параметры сетки
    'step': 2 * math.sqrt(S / math.pi) + 1,        # Шаг сетки (м)
    'offset_step': 0.5,                            # Шаг смещения (м)

    # Группировка данных
    'density_bins': [0, 0.01, 0.1, 0.2, float('inf')],  # 4 границы = 4 интервала
    'density_labels': ['0-0.01', '0.01-0.1', '0.1-0.2', '0.2+'],  # 4 метки для 4 интервалов
    'target_group': '0-0.01',             # Обновлено для соответствия количеству бинов

    # Системные настройки
    'output_crs': 'EPSG:3857'                      # Система координат для вывода
}

In [ ]:
# Комплексные игровые площадки площадью 900 м^2
S = 900  # Площадь в м^2
CONFIG = {
    'S': S,
    # Параметры обработки полигонов
    'inner_buffer': 40,                            # Величина внутреннего буфера (м)
    'buffer_radius': math.sqrt(S / math.pi),       # Радиус буфера вокруг точек (м)

    # Параметры сетки
    'step': 2 * math.sqrt(S / math.pi) + 1,        # Шаг сетки (м)
    'offset_step': 0.5,                            # Шаг смещения (м)

    # Группировка данных
    'density_bins': [0, 0.01, 0.1, 0.2, float('inf')],  # 4 границы = 4 интервала
    'density_labels': ['0-0.01', '0.01-0.1', '0.1-0.2', '0.2+'],  # 4 метки для 4 интервалов
    'target_group': '0-0.01',             # Обновлено для соответствия количеству бинов

    # Системные настройки
    'output_crs': 'EPSG:3857'                      # Система координат для вывода
}

In [ ]:
# Места для отдыха взрослых на расстоянии не меньше 30 м. друг от друга
CONFIG = {
    'S': S,
    # Параметры обработки полигонов
    'inner_buffer': 40,                            # Величина внутреннего буфера (м)
    'buffer_radius': 10,       # Радиус буфера вокруг точек (м)

    # Параметры сетки
    'step': 2 * math.sqrt(S / math.pi) + 1,        # Шаг сетки (м)
    'offset_step': 0.5,                            # Шаг смещения (м)

    # Группировка данных
    'density_bins': [0, 0.01, 0.1, 0.2, float('inf')],  # 4 границы = 4 интервала
    'density_labels': ['0-0.01', '0.01-0.1', '0.1-0.2', '0.2+'],  # 4 метки для 4 интервалов
    'target_group': '0-0.01',             # Обновлено для соответствия количеству бинов

    # Системные настройки
    'output_crs': 'EPSG:3857'                      # Система координат для вывода
}

Создание функций

In [ ]:
import time
import os
import numpy as np
import geopandas as gpd
import pandas as pd
from shapely.ops import unary_union
def process_group(group_gdf, output_dir, group_name, params):
    """Обработка группы с параметрами из конфига"""
    start_time = time.time()
    # Применяем внутренний буфер и фильтруем валидные полигоны
    valid_polygons = group_gdf.geometry.buffer(-params['inner_buffer'])
    valid_polygons = valid_polygons[~valid_polygons.is_empty]
    if valid_polygons.empty:
        print(f"Группа {group_name}: Нет валидных полигонов")
        return 0
    # Объединение полигонов и получение bbox
    union = unary_union(valid_polygons)
    bbox = union.bounds
    offsets = np.arange(0, params['step'], params['offset_step'])
    print(f"Обработка с параметрами: шаг={params['step']}м, радиус={params['buffer_radius']}м")
    total_files = 0
    # Основной цикл обработки
    for i, x_off in enumerate(offsets):
        for j, y_off in enumerate(offsets):
            x_coords = np.arange(bbox[0] + x_off, bbox[2], params['step'])
            y_coords = np.arange(bbox[1] + y_off, bbox[3], params['step'])
            if x_coords.size == 0 or y_coords.size == 0:
                continue
            # Генерация точек и фильтрация валидных
            xx, yy = np.meshgrid(x_coords, y_coords, indexing='xy')
            points = gpd.points_from_xy(xx.ravel(), yy.ravel(), crs=params['output_crs'])
            valid_points = points[points.buffer(params['buffer_radius']).within(union)]
            # Сохранение результатов
            if valid_points:
                filename = f"{i * len(offsets) + j}_{len(valid_points)}_point.gpkg"
                gpd.GeoDataFrame(geometry=valid_points).to_file(os.path.join(output_dir, filename))
                print(f"Сохранён {filename} | Точек: {len(valid_points)}")
                total_files += 1
    print(f"Группа {group_name} завершена за {time.time() - start_time:.1f} сек")
    return total_files
def process_single_range(input_path, output_dir, config):
    """Обрабатывает только указанный в config диапазон"""
    try:
        gdf = gpd.read_file(input_path)
        non_place = gdf[gdf['non-place1'] == 'non-place'].copy()
        # Проверка и группировка
        if len(config['density_bins']) != len(config['density_labels']) + 1:
            raise ValueError("Количество меток должно быть на 1 меньше количества границ")
        non_place['density_group'] = pd.cut(non_place['bench_density'], bins=config['density_bins'], labels=config['density_labels'])
        # Фильтрация и обработка
        target_data = non_place[non_place['density_group'] == config['target_group']]
        if not target_data.empty:
            total_files = process_group(target_data, output_dir, config['target_group'], params=config)
            print(f"\nИтого сохранено файлов: {total_files}")
    except Exception as e:
        print(f"Ошибка: {e}")

Применение

In [ ]:
if __name__ == '__main__':
    CUSTOM_CONFIG = CONFIG.copy()
    CUSTOM_CONFIG['target_group'] = '0-0.01'  # Пример: обработать только группу '0-0.01'
    input_path = "/content/output_b (2).gpkg"
    output_dir = "output_results"
    os.makedirs(output_dir, exist_ok=True)

    process_single_range(input_path, output_dir, CUSTOM_CONFIG)

Обработка с параметрами: шаг=34.851375012865375м, радиус=10м
Сохранён 0_96_point.gpkg | Точек: 96
Сохранён 1_91_point.gpkg | Точек: 91
Сохранён 2_92_point.gpkg | Точек: 92
Сохранён 3_93_point.gpkg | Точек: 93
Сохранён 4_91_point.gpkg | Точек: 91
Сохранён 5_92_point.gpkg | Точек: 92
Сохранён 6_90_point.gpkg | Точек: 90
Сохранён 7_92_point.gpkg | Точек: 92
Сохранён 8_89_point.gpkg | Точек: 89
Сохранён 9_91_point.gpkg | Точек: 91
Сохранён 10_91_point.gpkg | Точек: 91
Сохранён 11_90_point.gpkg | Точек: 90
Сохранён 12_90_point.gpkg | Точек: 90
Сохранён 13_90_point.gpkg | Точек: 90
Сохранён 14_92_point.gpkg | Точек: 92
Сохранён 15_93_point.gpkg | Точек: 93
Сохранён 16_90_point.gpkg | Точек: 90
Сохранён 17_93_point.gpkg | Точек: 93
Сохранён 18_90_point.gpkg | Точек: 90
Сохранён 19_89_point.gpkg | Точек: 89
Сохранён 20_88_point.gpkg | Точек: 88
Сохранён 21_88_point.gpkg | Точек: 88
Сохранён 22_88_point.gpkg | Точек: 88
Сохранён 23_87_point.gpkg | Точек: 87
Сохранён 24_89_point.gpkg | Точек: 89

KeyboardInterrupt: 